In [1]:
import pandas as pd
import json
from urllib.request import urlopen, quote,URLError, HTTPError
import requests
import time

# from urllib2 import Request, urlopen, URLError, HTTPError

In [2]:
data = pd.read_excel(r"D:\work\logistica\stations\JDStations\stations\website\XLSX\JDstations_zhejiang.xlsx")
data_city = pd.read_excel(r"D:\work\logistica\stations\JDStations\zhejiangcityID.xlsx")
province = "浙江省"

In [3]:
outpath = "D:\\work\\logistica\\stations\\JDStations\\stations\\"
# name_UTF8 = "JDstations_zhejiang_baidu.csv"
name_ANSI = "JDstations_zhejiang_baidu.xlsx"

In [4]:
print(data.columns)
print("*"*30)
print(data_city.columns)

Index(['cityId', 'countyId', 'provinceId', 'siteAddress', 'siteBusinessName',
       'siteLatitude', 'siteLongitude', 'siteName', 'siteTelephone'],
      dtype='object')
******************************
Index(['cityId', 'cityName'], dtype='object')


In [5]:
# df = data[['订单编号','网点地址']]
# df = data.copy()

In [6]:
df = pd.merge(data,data_city,how="left",on="cityId")

In [7]:
df.columns

Index(['cityId', 'countyId', 'provinceId', 'siteAddress', 'siteBusinessName',
       'siteLatitude', 'siteLongitude', 'siteName', 'siteTelephone',
       'cityName'],
      dtype='object')

In [8]:
def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式	
    ak = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak 
    req = urlopen(uri)
    res = req.read().decode() 
    temp = json.loads(res)
    return temp

In [9]:
print("地址去空前：{}".format(df['siteAddress'].isnull().value_counts()))
      
df = df.dropna(subset=["siteAddress"])
      


地址去空前：False    447
True       1
Name: siteAddress, dtype: int64


In [10]:
print("地址去空后：{}".format(df['siteAddress'].isnull().value_counts()))

地址去空后：False    447
Name: siteAddress, dtype: int64


In [11]:
df["siteAddress_all"] = province+ df['cityName'] + df['siteAddress']

In [12]:
df["siteAddress_all"]

0                       浙江省宁波市浙江省宁波市慈溪市逍林镇汽贸大道2119号
1                 浙江省宁波市浙江省宁波市慈溪市崇寿镇新兴西街73-75号（文广苑）
2             浙江省宁波市浙江省宁波市慈溪市掌起镇329国道掌起大街交叉口慈掌路363号
3                         浙江省宁波市浙江省宁波市慈溪市天元镇天兴路119号
4                           浙江省宁波市浙江省宁波市慈溪市新江北路957号
5                         浙江省宁波市浙江省宁波市慈溪市天元镇天兴路119号
6                          浙江省宁波市浙江宁波市慈溪市逍林镇林西路448号
7                          浙江省宁波市浙江宁波市慈溪市逍林镇林西路448号
8             浙江省宁波市浙江省宁波市慈溪市掌起镇329国道掌起大街交叉口慈掌路363号
9             浙江省宁波市浙江省宁波市慈溪市掌起镇329国道掌起大街交叉口慈掌路363号
10                      浙江省宁波市浙江省宁波市慈溪市逍林镇汽贸大道2119号
11                         浙江省宁波市浙江宁波市慈溪市逍林镇林西路448号
12                         浙江省宁波市浙江宁波市慈溪市逍林镇林西路448号
13                         浙江省宁波市浙江宁波市慈溪市逍林镇林西路448号
14                    浙江省宁波市浙江省宁波市慈溪市庵东镇七二三北大街12-4号
15     浙江省宁波市浙江省宁波市慈溪市观海卫镇世纪大道观海卫交警中队西侧（二师兄弟海鲜行正对面）
16                    浙江省宁波市浙江省宁波市慈溪市横河镇东上河村龙泉东路43号
17                      浙江省宁波市浙江省宁波市慈溪市逍林镇汽贸大道2119号
18     浙江省宁波市浙江省宁波市慈溪市观海卫镇世纪大道观海卫交警中队西侧（二师兄弟海鲜行正对面）
19          

In [13]:
start_time = time.time()
listA = []
for b in df['siteAddress_all']:
#     print(b)
    dictA = {}
    try:
        temp = getlnglat(b)
    except HTTPError as e:
        print("请求出错")
        pass
    else:    
        if ('result' in temp):
            dictA["siteLongitude_baidu"] = temp['result']['location']['lng']
            dictA["siteLatitude_baidu"] = temp['result']['location']['lat']
            dictA["sitePrecise_baidu"] = temp['result']['precise']
            dictA["siteConfidence_baidu"] = temp['result']['confidence']
            dictA["siteComprehension_baidu"] = temp['result']['comprehension']
            dictA["siteLevel_baidu"] = temp['result']['level']
        else:
            pass
    listA.append(dictA)
#     time.sleep(1)
end_time = time.time()
print("总用时：",end_time-start_time)

总用时： 15.920465230941772


0.857048749923706
5.161295413970947


In [14]:
listA

[{'siteComprehension_baidu': 96,
  'siteConfidence_baidu': 40,
  'siteLatitude_baidu': 30.168834878339283,
  'siteLevel_baidu': '道路',
  'siteLongitude_baidu': 121.3111557945222,
  'sitePrecise_baidu': 0},
 {'siteComprehension_baidu': 100,
  'siteConfidence_baidu': 75,
  'siteLatitude_baidu': 30.282068431382072,
  'siteLevel_baidu': '地产小区',
  'siteLongitude_baidu': 121.27786744245799,
  'sitePrecise_baidu': 1},
 {'siteComprehension_baidu': 99,
  'siteConfidence_baidu': 80,
  'siteLatitude_baidu': 30.124610262564367,
  'siteLevel_baidu': '道路',
  'siteLongitude_baidu': 121.46410218023483,
  'sitePrecise_baidu': 1},
 {'siteComprehension_baidu': 99,
  'siteConfidence_baidu': 80,
  'siteLatitude_baidu': 30.202946220989933,
  'siteLevel_baidu': '道路',
  'siteLongitude_baidu': 121.188515279568,
  'sitePrecise_baidu': 1},
 {'siteComprehension_baidu': 100,
  'siteConfidence_baidu': 80,
  'siteLatitude_baidu': 30.205171572481426,
  'siteLevel_baidu': '道路',
  'siteLongitude_baidu': 121.245483495585

In [15]:
df_A = pd.DataFrame(listA)

In [16]:
df_all = pd.concat([df,df_A],axis=1,join_axes=[df.index])

In [17]:

print("data:{}行".format(data.shape[0]))
print("df:{}行".format(df.shape[0]))
print("df_A:{}行".format(df_A.shape[0]))
print("df_all:{}行".format(df_all.shape[0]))

data:448行
df:447行
df_A:447行
df_all:447行


In [18]:
df_A.columns

Index(['siteComprehension_baidu', 'siteConfidence_baidu', 'siteLatitude_baidu',
       'siteLevel_baidu', 'siteLongitude_baidu', 'sitePrecise_baidu'],
      dtype='object')

In [19]:
df_all.columns

Index(['cityId', 'countyId', 'provinceId', 'siteAddress', 'siteBusinessName',
       'siteLatitude', 'siteLongitude', 'siteName', 'siteTelephone',
       'cityName', 'siteAddress_all', 'siteComprehension_baidu',
       'siteConfidence_baidu', 'siteLatitude_baidu', 'siteLevel_baidu',
       'siteLongitude_baidu', 'sitePrecise_baidu'],
      dtype='object')

In [20]:
df_all.to_excel(outpath + name_ANSI)
# df_all.to_csv(outpath + name_UTF8, index=False)